In [29]:
import numpy as np
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
import datetime as dt
import os
from functools import reduce
from fetch_symbols import get_symbols
import ccxt


class Data:
    def __init__(self, symbols, interval = '1h', start_time = dt.datetime(2020, 1, 1), end_time = dt.datetime(2020, 1, 2), get_data = True,
                 exchange = 'binance'):
        self.symbols = symbols
        self.interval = interval
        self.start_time = start_time
        self.end_time = end_time
        if exchange == 'binance':
            self.available_symbols = self.binance_symbols()
        elif exchange == 'kraken':
            self.available_symbols = self.kraken_symbols()
            
        if get_data:
            self.df = self.get_data()

    def binance_symbols(self):
        """Fetch available symbols from Binance API."""
        response = requests.get("https://api.binance.com/api/v3/exchangeInfo")
        exchange_info = response.json()
        valid_symbols = {s['symbol'] for s in exchange_info['symbols']}
        return [s for s in self.symbols if s in valid_symbols]
    
    def kraken_symbols(self):
        """Fetch available symbols from Kraken API."""
        exchange = ccxt.kraken()
        markets = exchange.load_markets()
        valid_symbols_ = {market['symbol'] for market in markets.values()}
        valid_symbols_ = [s.replace("/USD", "USD") for s in valid_symbols_ if s.endswith('USD')]
        valid_symbols_.sort()
        return [s for s in self.symbols if s in valid_symbols_]

    def fetch_symbol_data(self, symbol, date_list, url, limit):
        """Fetch kline data for a single symbol."""
        all_data = []
        for i in range(len(date_list) - 1):
            params = {
                'symbol': symbol,
                'interval': self.interval,
                'startTime': int(date_list[i].timestamp() * 1000),
                'endTime': int((date_list[i + 1] - dt.timedelta(seconds=1)).timestamp() * 1000),
                'limit': limit,
            }
            response = requests.get(url, params=params)
            data = response.json()
            if isinstance(data, list):
                all_data.extend(data)
        return symbol, all_data

    def get_binance_klines(self, limit=1000):
        """Fetch historical kline data for all symbols in parallel."""
        url = "https://api.binance.com/api/v3/klines"
        date_list = pd.date_range(start=self.start_time, end=self.end_time, freq='D').tolist()
        
        if not self.available_symbols[0].endswith('T'):
            self.available_symbols = [s + 'T' for s in self.available_symbols]

        print(self.available_symbols)
        # Use ThreadPoolExecutor for parallel fetching
        with ThreadPoolExecutor(max_workers=16) as executor:
            results = executor.map(
                lambda symbol: self.fetch_symbol_data(symbol, date_list, url, limit),
                self.available_symbols,
            )

        print(results)
        # Process and combine results
        data_frames = {}
        for symbol, data in results:
            if not data:
                continue
            df = pd.DataFrame(data)
            df = df.iloc[:, 0:6]
            df.columns = ['Open Time', 'open', 'high', 'low', 'close', 'volume']
            df.index = pd.to_datetime(df['Open Time'], unit='ms')
            df.drop('Open Time', axis=1, inplace=True)
            data_frames[symbol] = df

        if not data_frames:
            return None

        combined_df = pd.concat(data_frames, axis=1)
        combined_df = combined_df.swaplevel(axis=1).sort_index(axis=1)
        combined_df = combined_df.apply(pd.to_numeric, errors='coerce')

        return combined_df

    def prepare_data(self, df):
        """Prepare data for analysis."""
        _df = df.copy()
        for coin in df.columns.levels[1]:
            _df['returns', coin] = _df['close', coin].pct_change()
            _df['log_return', coin] = np.log(_df['returns', coin] + 1)
            _df["creturns", coin] = _df["log_return", coin].cumsum().apply(np.exp)
            _df['price', coin] = _df['close', coin]
            _df['volume_in_dollars', coin] = _df['close', coin] * _df['volume', coin]

        df = _df.stack(future_stack=True)
        df.sort_index(axis=1, inplace=True)
        df.index.names = ['date', 'coin']
        df.dropna(inplace=True)

        return df

    def upload_data(self, df, filename):
        """Save data to a CSV file."""
        df.to_csv(filename)

    def get_data(self):
        """Main function to fetch, prepare, and save data."""
        df = self.get_binance_klines()
        if df is not None:
            df = self.prepare_data(df)
            self.upload_data(df, 'data.csv')
        return df
    



class CSV_Data:
    def __init__(self, folder_path, symbols):
        self.folder_path = folder_path
        self.symbols = symbols
        self.df = self.process_folder(folder_path, symbols)
        self.df = self.prepare_data()
        self.upload_data_to_csv(self.df)
        
    
    def prepare_data(self):
        df = self.df.copy()
        for coin in df.columns.levels[1]:
            df['returns', coin] = df['close', coin].pct_change()
            df['log_return', coin] = np.log(df['returns', coin])
            df["creturns", coin] = df["log_return", coin].cumsum().apply(np.exp)
            df['price', coin] = df['close', coin]
            df['volume_in_dollars', coin] = df['close', coin] * df['volume', coin]

        df = df.stack(level=1, future_stack=True)
        df.sort_index(axis=1, inplace=True)
        df.index.names = ['date', 'coin']
        df.dropna(inplace=True)

        return df
    
    def get_data(self, file_path, symbols):
        df = pd.read_csv(file_path)
        df = df.drop(columns = df.columns[-1]).reset_index()
        df.drop(columns = df.columns[0], inplace = True)
        df.drop(index = 0, inplace = True)
        df.columns = ['date', 'coin', 'open', 'high', 'low', 'close', 'volume', 'volume_in_dollars']

        if not df['coin'].iloc[0] in symbols:
            return
        # Clean the date column by stripping whitespace
        df['date'] = df['date'].str.strip()
        # Parse the date column with mixed format
        df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
        
        df.set_index([df.columns[0], df.columns[1]], inplace = True)
        df = df.unstack()
        return df
    
    def process_folder(self, folder_path, symbols):
        # Get all CSV files in the folder
        csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
        
        dfs = []
        
        for file in csv_files:
            file_path = os.path.join(folder_path, file)
            df = self.get_data(file_path, symbols)
            if df is not None:
                dfs.append(df)
        

        # Get the union of all indices (dates) to align the data
        all_dates = reduce(pd.Index.union, [df.index.get_level_values(0) for df in dfs])

        # Reindex all DataFrames to the same set of dates (adding NaNs where data is missing)
        dfs_aligned = [df.reindex(all_dates, level=0, fill_value=None) for df in dfs]

        # Concatenate all DataFrames
        concatenated_df = pd.concat(dfs_aligned, axis=1)
        concatenated_df = concatenated_df.sort_index(axis=1)
        concatenated_df = concatenated_df.apply(pd.to_numeric, errors='coerce', downcast='float') #Essential to perform calculations
        
        return concatenated_df

    def upload_data_to_csv(self, df):
        # Upload the data to CSV file
        df.to_csv('all_data.csv')
    

    



# Example usage
# symbols = ['BTCUSD', 'ETHUSD']
# symbols = get_symbols()
# # Add the symbol to each string in the list
# updated_symbols = [s + 'T' for s in symbols]
# interval = '1h'
# start_time = dt.datetime(2020, 1, 1)
# end_time = dt.datetime(2020, 1, 7)
# df = Data(updated_symbols, interval, start_time, end_time).df
# print(df)


#Use the below for uploading full data (uploaded to csv)
# symbols = get_symbols()
# binance_symbols = Data(symbols)
# folder_path = r'C:\Users\yassi\OneDrive\Documents\Trading\Algo Trading Projects\Algo Business\data\Binance Data (CSV)'
# df = CSV_Data(folder_path, symbols).df

In [3]:
test_symbols = ['BTCUSD', 'ETHUSD']

In [4]:
data_instance = Data(symbols, exchange = 'kraken')

['BTCUSDT', 'ETHUSDT']
<generator object Executor.map.<locals>.result_iterator at 0x000002B96A147C40>


In [8]:
symbols = get_symbols()

In [9]:
symbols

['BTCUSD',
 'ETHUSD',
 'BNBUSD',
 'SOLUSD',
 'ADAUSD',
 'TRXUSD',
 'SUIUSD',
 'AVAXUSD',
 'TONUSD',
 'HBARUSD',
 'BCHUSD',
 'HYPEUSD',
 'NEARUSD',
 'APTUSD',
 'ICPUSD',
 'CROUSD',
 'OMUSD',
 'VETUSD',
 'XMRUSD',
 'TAOUSD',
 'FILUSD',
 'ALGOUSD',
 'KASUSD',
 'TIAUSD',
 'SUSD',
 'THETAUSD',
 'INJUSD',
 'FTMUSD',
 'SEIUSD',
 'FTNUSD',
 'GALAUSD',
 'XDCUSD',
 'FLRUSD',
 'XTZUSD',
 'KAIAUSD',
 'IOTAUSD',
 'FLOWUSD',
 'ARUSD',
 'AIOZUSD',
 'DYDXUSD',
 'COREUSD',
 'EGLDUSD',
 'CHZUSD',
 'XECUSD',
 'RONUSD',
 'GNOUSD',
 'KAVAUSD',
 'ROSEUSD',
 'ASTRUSD',
 'VANAUSD',
 'VRSCUSD',
 'ENJUSD',
 'CELOUSD',
 'WEMIXUSD',
 'ZETAUSD',
 'ELFUSD',
 'PEAQUSD',
 'POLYXUSD',
 'XCHUSD',
 'QUBICUSD',
 'ONEUSD',
 'KDAUSD',
 'DCRUSD',
 'HEARTUSD',
 'ZANOUSD',
 'SKLUSD',
 'HIVEUSD',
 'CSPRUSD',
 'GLMRUSD',
 'XRDUSD',
 'SUPRAUSD',
 'VANRYUSD',
 'DAGUSD',
 'RBNTUSD',
 'ALEOUSD',
 'CHRUSD',
 'ORAIUSD',
 'WAXPUSD',
 'ONGUSD',
 'BBUSD',
 'SAGAUSD',
 'ERGUSD',
 'ALPHUSD',
 'MOVRUSD',
 'DUSKUSD',
 'OMNIUSD',
 'AZEROUSD'

In [3]:
import pandas as pd

In [4]:
file = 'all_data.csv'

all_data = pd.read_csv(file, index_col=['date', 'coin'], parse_dates=['date'])

In [10]:
all_data.index = all_data.index.set_levels(
    all_data.index.levels[1].str.replace('USDT', '/USD', regex=False), level='coin'
)

In [11]:
all_data

close      creturns         high  \
date                coin                                               
2017-08-17 05:00:00 BTC/USD   4315.320000  1.506209e-03  4328.690000   
                    ETH/USD    303.100006  4.940270e-03   303.279999   
2017-08-17 06:00:00 BTC/USD   4324.350000  3.151810e-06  4345.450000   
2017-08-17 07:00:00 BTC/USD   4349.990000  1.868776e-08  4349.990000   
                    ETH/USD    307.959991  8.617874e-05   307.959991   
...                                   ...           ...          ...   
2024-12-27 23:00:00 SYS/USD      0.112200  0.000000e+00     0.112400   
                    TRX/USD      0.259400  0.000000e+00     0.259600   
                    VET/USD      0.045760  0.000000e+00     0.045760   
                    WAXP/USD     0.040910  0.000000e+00     0.040940   
                    XEC/USD      0.000035  0.000000e+00     0.000035   

                              log_return          low         open  \
date                coin                                             
2017-08-17 05:00:00 BTC/USD    -6.498159  4291.370000  4308.830000   
                    ETH/USD    -5.310335   300.000000   301.609985   
2017-08-17 06:00:00 BTC/USD    -6.169374  4309.370000  4330.290000   
2017-08-17 07:00:00 BTC/USD    -5.127863  4287.410000  4316.620000   
                    ETH/USD    -4.048752   302.600006   302.679993   
...                                  ...          ...          ...   
2024-12-27 23:00:00 SYS/USD    -5.632999     0.111100     0.111600   
                    TRX/USD    -7.860292     0.258600     0.259400   
                    VET/USD    -5.479996     0.045370     0.045580   
                    WAXP/USD   -5.915972     0.040590     0.040770   
                    XEC/USD    -5.503282     0.000034     0.000034   

                                    price   returns        volume  \
date                coin                                            
2017-08-17 05:00:00 BTC/USD   4315.320000  0.001506  2.323492e+01   
                    ETH/USD    303.100006  0.004940  3.776725e+02   
2017-08-17 06:00:00 BTC/USD   4324.350000  0.002093  7.229691e+00   
2017-08-17 07:00:00 BTC/USD   4349.990000  0.005929  4.443249e+00   
                    ETH/USD    307.959991  0.017444  7.547451e+02   
...                                   ...       ...           ...   
2024-12-27 23:00:00 SYS/USD      0.112200  0.003578  7.290790e+05   
                    TRX/USD      0.259400  0.000386  1.499596e+07   
                    VET/USD      0.045760  0.004169  1.004372e+07   
                    WAXP/USD     0.040910  0.002696  1.056349e+06   
                    XEC/USD      0.000035  0.004073  1.147830e+09   

                              volume_in_dollars  
date                coin                         
2017-08-17 05:00:00 BTC/USD        1.002661e+05  
                    ETH/USD        1.144725e+05  
2017-08-17 06:00:00 BTC/USD        3.126371e+04  
2017-08-17 07:00:00 BTC/USD        1.932809e+04  
                    ETH/USD        2.324313e+05  
...                                         ...  
2024-12-27 23:00:00 SYS/USD        8.180266e+04  
                    TRX/USD        3.889953e+06  
                    VET/USD        4.596008e+05  
                    WAXP/USD       4.321524e+04  
                    XEC/USD        3.961160e+04  

[956964 rows x 10 columns]

In [ ]:
#Choose the last X hours of data
cutoff_time = dt.datetime.now() - dt.timedelta(hours=2201)

# Filter rows where 'date' is within the last 2201 hours
new_all_data = all_data.loc[all_data.index.get_level_values('date') >= cutoff_time]

In [22]:
new_all_data

close  creturns          high  \
date                coin                                             
2024-10-12 20:00:00 ADA/USD       0.352700       0.0      0.353100   
                    ALGO/USD      0.123000       0.0      0.123100   
                    AR/USD       19.590000       0.0     19.650000   
                    BTC/USD   63043.290000       0.0  63113.210000   
                    CELO/USD      0.811500       0.0      0.813100   
...                                    ...       ...           ...   
2024-12-27 23:00:00 SYS/USD       0.112200       0.0      0.112400   
                    TRX/USD       0.259400       0.0      0.259600   
                    VET/USD       0.045760       0.0      0.045760   
                    WAXP/USD      0.040910       0.0      0.040940   
                    XEC/USD       0.000035       0.0      0.000035   

                              log_return           low          open  \
date                coin                                               
2024-10-12 20:00:00 ADA/USD    -7.474382      0.351500      0.352400   
                    ALGO/USD   -6.420012      0.122700      0.122700   
                    AR/USD     -5.272477     19.490000     19.490000   
                    BTC/USD    -7.283200  63000.000000  63000.010000   
                    CELO/USD   -4.373827      0.801500      0.801500   
...                                  ...           ...           ...   
2024-12-27 23:00:00 SYS/USD    -5.632999      0.111100      0.111600   
                    TRX/USD    -7.860292      0.258600      0.259400   
                    VET/USD    -5.479996      0.045370      0.045580   
                    WAXP/USD   -5.915972      0.040590      0.040770   
                    XEC/USD    -5.503282      0.000034      0.000034   

                                     price   returns        volume  \
date                coin                                             
2024-10-12 20:00:00 ADA/USD       0.352700  0.000567  1.331361e+06   
                    ALGO/USD      0.123000  0.001629  5.106250e+05   
                    AR/USD       19.590000  0.005131  1.530563e+04   
                    BTC/USD   63043.290000  0.000687  2.079624e+02   
                    CELO/USD      0.811500  0.012603  2.238935e+05   
...                                    ...       ...           ...   
2024-12-27 23:00:00 SYS/USD       0.112200  0.003578  7.290790e+05   
                    TRX/USD       0.259400  0.000386  1.499596e+07   
                    VET/USD       0.045760  0.004169  1.004372e+07   
                    WAXP/USD      0.040910  0.002696  1.056349e+06   
                    XEC/USD       0.000035  0.004073  1.147830e+09   

                              volume_in_dollars  
date                coin                         
2024-10-12 20:00:00 ADA/USD        4.695712e+05  
                    ALGO/USD       6.280688e+04  
                    AR/USD         2.998373e+05  
                    BTC/USD        1.311063e+07  
                    CELO/USD       1.816896e+05  
...                                         ...  
2024-12-27 23:00:00 SYS/USD        8.180266e+04  
                    TRX/USD        3.889953e+06  
                    VET/USD        4.596008e+05  
                    WAXP/USD       4.321524e+04  
                    XEC/USD        3.961160e+04  

[41976 rows x 10 columns]

In [23]:
new_all_data.to_csv('market_data.csv')

In [31]:
data_instance = Data(symbols, exchange = 'kraken', get_data = False)
available_symbols  = data_instance.available_symbols

In [32]:
available_symbols

['BTCUSD',
 'ETHUSD',
 'SOLUSD',
 'ADAUSD',
 'TRXUSD',
 'SUIUSD',
 'AVAXUSD',
 'TONUSD',
 'BCHUSD',
 'NEARUSD',
 'APTUSD',
 'ICPUSD',
 'XMRUSD',
 'TAOUSD',
 'FILUSD',
 'ALGOUSD',
 'KASUSD',
 'TIAUSD',
 'INJUSD',
 'FTMUSD',
 'SEIUSD',
 'GALAUSD',
 'FLRUSD',
 'XTZUSD',
 'FLOWUSD',
 'DYDXUSD',
 'EGLDUSD',
 'CHZUSD',
 'GNOUSD',
 'KAVAUSD',
 'ASTRUSD',
 'ENJUSD',
 'GLMRUSD',
 'VANRYUSD',
 'CHRUSD',
 'SAGAUSD',
 'MOVRUSD',
 'OMNIUSD',
 'KUJIUSD',
 'RENDERUSD',
 'TAOUSD',
 'FILUSD',
 'GRTUSD',
 'JASMYUSD',
 'BTTUSD',
 'HNTUSD',
 'AKTUSD',
 'LPTUSD',
 'TRACUSD',
 'ANKRUSD',
 'SCUSD',
 'ATHUSD',
 'PHAUSD',
 'NOSUSD',
 'HONEYUSD',
 'RLCUSD',
 'DENTUSD',
 'OCEANUSD',
 'STORJUSD',
 'LMWRUSD',
 'NYMUSD',
 'OXTUSD',
 'RADUSD',
 'BLZUSD',
 'NODLUSD',
 'XRTUSD',
 'MXCUSD',
 'BTCUSD',
 'DOGEUSD',
 'BCHUSD',
 'LTCUSD',
 'ETCUSD',
 'XMRUSD',
 'KASUSD',
 'ZECUSD',
 'DASHUSD',
 'SCUSD',
 'ETHWUSD',
 'BEAMUSD',
 'OMNIUSD',
 'SAFEUSD',
 'ETHUSD',
 'SOLUSD',
 'ADAUSD',
 'AVAXUSD',
 'TONUSD',
 'DOTUSD',
 'NEAR

In [ ]:
halal_symbols = []